# TensorBoard - callback

In [1]:
import os
from glob import glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import layers, datasets 
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data Preprocess

In [5]:
train_paths = glob('./dataset/cifar/train/*.png')[:1000]
test_paths = glob('./dataset/cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    
    # 이번에는 onehot이 아닌 label 번호로
    return tf.argmax(onehot)

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Callbacks

In [14]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [15]:
tensorboard = TensorBoard(log_dir=logdir,
                          write_graph=True, 
                          write_images=True,
                          histogram_freq=1
)

In [16]:
%tensorboard --logdir logs --port 8008

Reusing TensorBoard on port 8008 (pid 38316), started 0:24:34 ago. (Use '!kill 38316' to kill it.)

## Training

In [17]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit(train_dataset,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps,
          epochs=num_epochs,
          callbacks=[tensorboard]
)

Epoch 1/10
31/31 [==============================] - 2s 49ms/step - loss: 2.3166 - accuracy: 0.1085 - val_loss: 2.2775 - val_accuracy: 0.1129
Epoch 2/10
31/31 [==============================] - 1s 37ms/step - loss: 2.2584 - accuracy: 0.1351 - val_loss: 2.2499 - val_accuracy: 0.1603
Epoch 3/10
31/31 [==============================] - 1s 38ms/step - loss: 2.2109 - accuracy: 0.1780 - val_loss: 2.1012 - val_accuracy: 0.1905
Epoch 4/10
31/31 [==============================] - 1s 36ms/step - loss: 2.1512 - accuracy: 0.1849 - val_loss: 2.0784 - val_accuracy: 0.2470
Epoch 5/10
31/31 [==============================] - 1s 37ms/step - loss: 2.1150 - accuracy: 0.1973 - val_loss: 2.0213 - val_accuracy: 0.2278
Epoch 6/10
31/31 [==============================] - 1s 38ms/step - loss: 2.0814 - accuracy: 0.2510 - val_loss: 2.0362 - val_accuracy: 0.2329
Epoch 7/10
31/31 [==============================] - 1s 37ms/step - loss: 2.0289 - accuracy: 0.2314 - val_loss: 2.0073 - val_accuracy: 0.2611
Epoch 8/10
31

### LambdaCallback

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

In [18]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [19]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [20]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [21]:
# ignore : RuntimeWarning: invalid value encountered in true_divide
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [22]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit(train_dataset,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps,
          epochs=num_epochs,
          callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
31/31 [==============================] - 2s 59ms/step - loss: 1.8757 - accuracy: 0.2924 - val_loss: 1.9582 - val_accuracy: 0.2742
Epoch 2/10
31/31 [==============================] - 2s 51ms/step - loss: 1.8678 - accuracy: 0.2934 - val_loss: 1.8242 - val_accuracy: 0.3367
Epoch 3/10
31/31 [==============================] - 2s 49ms/step - loss: 1.7943 - accuracy: 0.3244 - val_loss: 1.8015 - val_accuracy: 0.3296
Epoch 4/10
31/31 [==============================] - 2s 53ms/step - loss: 1.7899 - accuracy: 0.3337 - val_loss: 1.8286 - val_accuracy: 0.3024
Epoch 5/10
31/31 [==============================] - 2s 52ms/step - loss: 1.7684 - accuracy: 0.3264 - val_loss: 1.8473 - val_accuracy: 0.3438
Epoch 6/10
31/31 [==============================] - 2s 55ms/step - loss: 1.7571 - accuracy: 0.3523 - val_loss: 1.7518 - val_accuracy: 0.3679
Epoch 7/10
31/31 [==============================] - 2s 53ms/step - loss: 1.6919 - accuracy: 0.3833 - val_loss: 1.7096 - val_accuracy: 0.3629
Epoch 8/10
31

## Expert

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [24]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [25]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
        
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [26]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [27]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [28]:
file_writer = tf.summary.create_file_writer(logdir)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step)
        tf.summary.scalar('loss', train_loss.result(), step=step)

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

```
Terminal:
$ tensorboard --logdir=logs
```